In [1]:
import sys
import nltk
import torch.nn.functional as F

import torch
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
sys.path.insert(0, '..')
import torch.nn as nn
import numpy as np
import torch
import pickle



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\gxb18167\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:



print(torch.__version__)
print("GPU Available:", torch.cuda.is_available())

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = "cpu"

2.0.1
GPU Available: False


In [12]:
batch_size = 64
word_embedding_dim = 50
output_shape = (1, 105, 8)
torch.manual_seed(1)
np.random.seed(1)

In [13]:


# To load the lists from the file:
with open(r'C:\Users\gxb18167\PycharmProjects\SIGIR_EEG_GAN\Development\EEG-To-Text-GAN\Data\EEG_Text_Pairs.pkl', 'rb') as file:
    EEG_word_level_embeddings = pickle.load(file)
    EEG_word_level_labels = pickle.load(file)

In [18]:

has_inf = np.isinf(EEG_word_level_embeddings).any()
std_dev = np.std(EEG_word_level_embeddings)

print("Standard Deviation:", std_dev)


In [16]:
EEG_word_level_embeddings = np.array(EEG_word_level_embeddings)

In [17]:
torch.isnan(torch.from_numpy(EEG_word_level_embeddings)).any()

In [18]:
missing_value_indices = np.isnan(EEG_word_level_embeddings)

# Print indices and values of missing values
print("Indices of Missing Values:")
for i in range(EEG_word_level_embeddings.shape[0]):
    for j in range(EEG_word_level_embeddings.shape[1]):
        for k in range(EEG_word_level_embeddings.shape[2]):
            if missing_value_indices[i, j, k]:
                print(f"Index: ({i}, {j}, {k}), Value: {EEG_word_level_embeddings[i, j, k]}")

In [19]:
def create_word_label_embeddings(Word_Labels_List):
    tokenized_words = []
    for i in range(len(Word_Labels_List)):
        tokenized_words.append([Word_Labels_List[i]])
    model = Word2Vec(sentences=tokenized_words, vector_size=word_embedding_dim, window=5, min_count=1, workers=4)
    word_embeddings = {word: model.wv[word] for word in model.wv.index_to_key}
    print("Number of word embeddings:", len(word_embeddings))
    #word, embedding = list(word_embeddings.items())[10]
    #print(f"Word: {word}, Embedding: {embedding}")

    Embedded_Word_labels = []
    for word in EEG_word_level_labels:
        Embedded_Word_labels.append(word_embeddings[word])

    return Embedded_Word_labels, word_embeddings

def create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels):
    EEG_word_level_embeddings_normalize = (EEG_word_level_embeddings - np.mean(EEG_word_level_embeddings)) / np.std(EEG_word_level_embeddings)

    float_tensor = torch.tensor(EEG_word_level_embeddings_normalize, dtype=torch.float)
    float_tensor = float_tensor.unsqueeze(1)

    print(EEG_word_level_embeddings_normalize)
    # Calculate mean and standard deviation
    print(torch.isnan(float_tensor).any())

    train_data = []
    for i in range(len(float_tensor)):
       train_data.append([float_tensor[i], Embedded_Word_labels[i]])
    trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=64)
    return trainloader

In [20]:
Embedded_Word_labels, word_embeddings = create_word_label_embeddings(EEG_word_level_labels)
trainloader = create_dataloader(EEG_word_level_embeddings, Embedded_Word_labels)

Number of word embeddings: 2397
[[[-1.02536569 -0.8012387  -0.89344778 ... -0.78901025 -1.02923809
   -0.90870677]
  [-0.87301889 -0.78214638 -0.62023623 ... -0.84317268 -0.82236941
   -0.82083052]
  [-0.95071075 -0.88973399 -0.77370027 ... -0.82746559 -0.53630228
   -0.54953861]
  ...
  [-0.25248797  0.61655852  0.367149   ... -0.54500524 -0.59043094
   -0.49396242]
  [ 0.35400258 -0.59136324 -0.59922374 ... -0.54399227 -0.4733795
   -0.72263964]
  [-0.66811198 -0.59378567 -0.40498261 ... -0.83837453 -0.55229494
   -1.0841225 ]]

 [[-1.04352227 -0.81013303 -0.96217312 ... -0.90306235 -0.99172719
   -0.9384617 ]
  [-0.90340717 -0.80319306 -0.81768235 ... -0.88755351 -0.99444696
   -1.01240863]
  [-0.96001894 -0.91794949 -0.9211666  ... -1.01422661 -0.72809198
   -0.7240152 ]
  ...
  [-0.16663774  0.777237    0.50666568 ... -0.70147193 -0.61978433
   -0.66760818]
  [ 0.4092072  -0.75888738 -0.61551424 ... -0.40844637 -0.41022822
   -0.83054213]
  [-0.82071962 -0.73833184 -0.40159305 ...

In [21]:
#sanity check:
i1, l1 = next(iter(trainloader))
print(i1.shape, l1)

torch.Size([64, 1, 105, 8]) tensor([[-0.0104, -0.0148, -0.0058,  ..., -0.0054,  0.0076,  0.0007],
        [-0.0143,  0.0025, -0.0144,  ...,  0.0004,  0.0095, -0.0142],
        [-0.0190,  0.0144,  0.0181,  ..., -0.0012,  0.0017, -0.0025],
        ...,
        [-0.0192,  0.0179,  0.0083,  ...,  0.0083, -0.0085, -0.0077],
        [ 0.0036,  0.0141,  0.0059,  ..., -0.0067,  0.0032,  0.0032],
        [-0.0179,  0.0013, -0.0128,  ..., -0.0100,  0.0083,  0.0113]])


C:\Users\gxb18167\Anaconda3\lib\site-packages\torch\utils\data\_utils\collate.py:171: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\b\abs_abjetg6_iu\croot\pytorch_1686932924616\work\torch\csrc\utils\tensor_numpy.cpp:212.)
  return collate([torch.as_tensor(b) for b in batch], collate_fn_map=collate_fn_map)


In [22]:
z_size = 100
image_size = (105, 8)

n_filters = 32

In [23]:
## Loss function and optimizers:
loss_fn = nn.BCELoss()

In [24]:
def create_noise(batch_size, z_size, mode_z):
    if mode_z == 'uniform':
        input_z = torch.rand(batch_size, z_size)*2 - 1
    elif mode_z == 'normal':
        input_z = torch.randn(batch_size, z_size)
    return input_z

mode_z = 'uniform'
fixed_z = create_noise(batch_size, z_size, mode_z).to(device)

def create_samples(g_model, input_z, input_t):
    g_output = g_model(input_z, input_t)
    images = torch.reshape(g_output, (batch_size, *image_size))
    return (images+1)/2.0

In [25]:
noise = create_noise(64, 100, "uniform")

In [26]:
noise.shape

torch.Size([64, 100])

In [57]:
class Generator(nn.Module):
    def __init__(self, noise_dim, word_embedding_dim, output_shape):
        super(Generator, self).__init__()

        self.noise_dim = noise_dim
        self.word_embedding_dim = word_embedding_dim

        # Define the layers of your generator
        self.fc_noise = nn.Linear(noise_dim, 105 * 8)
        self.fc_word_embedding = nn.Linear(word_embedding_dim, 105 * 8)
        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(64, 1, kernel_size=3, stride=1, padding=1)



    def forward(self, noise, word_embedding):
        # Process noise
        noise = self.fc_noise(noise)
        noise = noise.view(noise.size(0), 1, 105, 8)

        # Process word embedding
        word_embedding = self.fc_word_embedding(word_embedding)
        word_embedding = word_embedding.view(word_embedding.size(0), 1, 105, 8)

        # Concatenate noise and word embedding
        combined_input = torch.cat([noise, word_embedding], dim=1)

        # Upsample and generate the output
        z = self.conv1(combined_input)
        z = self.bn1(z)
        z = self.relu(z)
        z = self.conv2(z)

        return z

class DiscriminatorWGAN(nn.Module):
    def __init__(self, n_filters):
        super(DiscriminatorWGAN, self).__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1, n_filters, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters, n_filters*2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(n_filters * 2),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters*2, n_filters*4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.InstanceNorm2d(n_filters*4),
            nn.LeakyReLU(0.2),

            nn.Conv2d(n_filters*4, 1, kernel_size=3, stride=1, padding=1, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        output = self.network(input)
        return output.view(-1, 1)

In [58]:
gen_model = Generator(z_size, word_embedding_dim, output_shape).to(device)
disc_model = DiscriminatorWGAN(n_filters).to(device)

g_optimizer = torch.optim.Adam(gen_model.parameters(), 0.00002)
d_optimizer = torch.optim.Adam(disc_model.parameters(), 0.00002)

In [59]:
from torch.autograd import grad as torch_grad


def gradient_penalty(real_data, generated_data):
    batch_size = real_data.size(0)

    # Calculate interpolation
    alpha = torch.rand(real_data.shape[0], 1, 1, 1, requires_grad=True, device=device)
    #print("Gen:", generated_data.shape)
    interpolated = alpha * real_data + (1 - alpha) * generated_data

    # Calculate probability of interpolated examples
    proba_interpolated = disc_model(interpolated)

    # Calculate gradients of probabilities with respect to examples
    gradients = torch_grad(outputs=proba_interpolated, inputs=interpolated,
                           grad_outputs=torch.ones(proba_interpolated.size(), device=device),
                           create_graph=True, retain_graph=True)[0]

    gradients = gradients.view(batch_size, -1)
    gradients_norm = gradients.norm(2, dim=1)
    return lambda_gp * ((gradients_norm - 1)**2).mean()


In [69]:
## Train the discriminator
def d_train_wgan(x, input_t):
    disc_model.zero_grad()

    batch_size = x.size(0)
    x = x.to(device)
    print("X:", x.shape)
    # Calculate probabilities on real and generated data
    d_real = disc_model(x)
    input_z = create_noise(batch_size, z_size, mode_z).to(device)
    g_output = gen_model(input_z, input_t)


    d_generated = disc_model(g_output)
    print("G output:", g_output.shape)

    d_loss = d_generated.mean() - d_real.mean() + gradient_penalty(x.data, g_output.data)

    d_loss.backward()
    d_optimizer.step()

    return d_loss.data.item()

In [70]:
## Train the generator
def g_train_wgan(x, input_t):
    gen_model.zero_grad()

    batch_size = x.size(0)
    input_z = create_noise(batch_size, z_size, mode_z).to(device)

    g_output = gen_model(input_z, input_t)

    d_generated = disc_model(g_output)
    g_loss = -d_generated.mean()

    # gradient backprop & optimize ONLY G's parameters
    g_loss.backward()
    g_optimizer.step()

    return g_loss.data.item()

In [71]:
epoch_samples_wgan = []
lambda_gp = 10.0
num_epochs = 100
torch.manual_seed(1)
critic_iterations = 5

for epoch in range(1, num_epochs+1):
    gen_model.train()
    d_losses, g_losses = [], []
    for i, (x, t) in enumerate(trainloader):
        #print("T:", t)
        for _ in range(critic_iterations):
            d_loss = d_train_wgan(x, t)
            print("D Loss:", d_loss)
        d_losses.append(d_loss)
        g_losses.append(g_train_wgan(x, t))

    print(f'Epoch {epoch:03d} | D Loss >>'
          f' {torch.FloatTensor(d_losses).mean():.4f}')
    gen_model.eval()
    epoch_samples_wgan.append(
        create_samples(gen_model, fixed_z, t).detach().cpu().numpy())

G loss: tensor(-0.3404, grad_fn=<NegBackward0>)
G loss: tensor(-0.3614, grad_fn=<NegBackward0>)
G loss: tensor(-0.3560, grad_fn=<NegBackward0>)
G loss: tensor(-0.3219, grad_fn=<NegBackward0>)
G loss: tensor(-0.2841, grad_fn=<NegBackward0>)
G loss: tensor(-0.3415, grad_fn=<NegBackward0>)
G loss: tensor(-0.3050, grad_fn=<NegBackward0>)
G loss: tensor(-0.3248, grad_fn=<NegBackward0>)
G loss: tensor(-0.3082, grad_fn=<NegBackward0>)
G loss: tensor(-0.3295, grad_fn=<NegBackward0>)
G loss: tensor(-0.3386, grad_fn=<NegBackward0>)
G loss: tensor(-0.3445, grad_fn=<NegBackward0>)
G loss: tensor(-0.3099, grad_fn=<NegBackward0>)
G loss: tensor(-0.3386, grad_fn=<NegBackward0>)
G loss: tensor(-0.3416, grad_fn=<NegBackward0>)
G loss: tensor(-0.3232, grad_fn=<NegBackward0>)
G loss: tensor(-0.3469, grad_fn=<NegBackward0>)
G loss: tensor(-0.3702, grad_fn=<NegBackward0>)
G loss: tensor(-0.3191, grad_fn=<NegBackward0>)
G loss: tensor(-0.3288, grad_fn=<NegBackward0>)
G loss: tensor(-0.3053, grad_fn=<NegBack

KeyboardInterrupt: 